In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

ettH1_data = pd.read_csv('ETTh1.csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')

# Preprocess ETT H1 data
ett_h1 = ettH1_data[['date', 'HUFL']]
ett_h1['date'] = pd.to_datetime(ett_h1['date'])
ett_h1 = ett_h1.set_index('date')  # Use .set_index() and update the DataFrame

# Preprocess ETT H2 data
ett_h2 = ettH2_data[['date', 'HULL']]
ett_h2['date'] = pd.to_datetime(ett_h2['date'])
ett_h2 = ett_h2.set_index('date')  # Use .set_index() and update the DataFrame

# Preprocess ETT M1 data
ett_m1 = ettM1_data[['date', 'HUFL']]
ett_m1['date'] = pd.to_datetime(ett_m1['date'])
ett_m1 = ett_m1.set_index('date')  # Use .set_index() and update the DataFrame

# Preprocess ETT M2 data
ett_m2 = ettM2_data[['date', 'HULL']]
ett_m2['date'] = pd.to_datetime(ett_m2['date'])
ett_m2 = ett_m2.set_index('date')  # Use .set_index() and update the DataFrame

# Scale the data using MinMaxScaler
scaler_h1 = MinMaxScaler()
scaler_h2 = MinMaxScaler()
scaler_m1 = MinMaxScaler()
scaler_m2 = MinMaxScaler()

ett_h1_scaled = scaler_h1.fit_transform(ett_h1_values)
ett_h2_scaled = scaler_h2.fit_transform(ett_h2_values)
ett_m1_scaled = scaler_m1.fit_transform(ett_m1_values)
ett_m2_scaled = scaler_m2.fit_transform(ett_m2_values)


C:\Users\ashis\AppData\Local\Temp\ipykernel_10688\2600878127.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ett_h1['date'] = pd.to_datetime(ett_h1['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_10688\2600878127.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ett_h2['date'] = pd.to_datetime(ett_h2['date'])
C:\Users\ashis\AppData\Local\Temp\ipykernel_10688\2600878127.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [30]:
def create_lstm_model(sequence_length):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm(data):
    sequence_length = 50
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), 0])
        y.append(data[i + sequence_length, 0])
    X, y = np.array(X), np.array(y)

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = create_lstm_model(sequence_length)
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, 1))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)

predictions_etth1 = train_lstm(ett_h1_scaled)
predictions_etth2 = train_lstm(ett_h2_scaled)
predictions_ettm1 = train_lstm(ett_m1_scaled)
predictions_ettm2 = train_lstm(ett_m2_scaled)

combined_matrix = np.column_stack((predictions_etth1, predictions_etth2, predictions_ettm1, predictions_ettm2))
print(combined_matrix)


Epoch 1/5
543/543 [==============================] - 26s 40ms/step - loss: 0.0151
Epoch 2/5
543/543 [==============================] - 22s 41ms/step - loss: 0.0039
Epoch 3/5
543/543 [==============================] - 20s 37ms/step - loss: 0.0029
Epoch 4/5
543/543 [==============================] - 19s 36ms/step - loss: 0.0028
Epoch 5/5
1/1 [==============================] - 0s 28ms/step
Epoch 1/5
543/543 [==============================] - 23s 35ms/step - loss: 0.0034
Epoch 2/5
543/543 [==============================] - 20s 37ms/step - loss: 0.0012
Epoch 3/5
543/543 [==============================] - 18s 33ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 18s 33ms/step - loss: 9.8624e-04
Epoch 5/5
1/1 [==============================] - 0s 17ms/step
Epoch 1/5
2176/2176 [==============================] - 87s 38ms/step - loss: 0.0027
Epoch 2/5
2176/2176 [==============================] - 104s 48ms/step - loss: 0.0011
Epoch 3/5
2176/2176 [==========================